In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import numpy as np
import pandas as pd
from keras.preprocessing.text import Tokenizer
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model
from keras.layers import Conv1D, GlobalMaxPool1D, Input
from keras.layers import Dense, Dropout, MaxPooling1D
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split



data = pd.read_csv('drive/My Drive/jigsaw-toxic-comment-classification-challenge/train.csv')

In [0]:
#print(data)
data = data.values
X_train = data[:,1]
Y_train = data[:,2]

print(Y_train)

[0 0 0 ... 0 0 0]


In [0]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
encoded = tokenizer.texts_to_sequences(X_train)

print(X_train[8])
print(encoded[8:9])

Sorry if the word 'nonsense' was offensive to you. Anyway, I'm not intending to write anything in the article(wow they would jump on me for vandalism), I'm merely requesting that it be more encyclopedic so one can use it for school as a reference. I have been to the selective breeding page but it's almost a stub. It points to 'animal breeding' which is a short messy article that gives you no info. There must be someone around with expertise in eugenics? 93.161.107.169
[[263, 22, 1, 286, 34847, 24, 1476, 2, 6, 489, 73, 14, 9215, 2, 330, 215, 10, 1, 23, 1473, 51, 47, 2898, 15, 35, 12, 199, 73, 1051, 1060, 9, 11, 16, 57, 1390, 37, 48, 40, 81, 11, 12, 480, 17, 5, 273, 7, 18, 55, 2, 1, 4850, 8153, 27, 26, 78, 669, 5, 1368, 11, 687, 2, 96147, 96148, 53, 8, 5, 702, 7774, 23, 9, 1094, 6, 46, 468, 41, 251, 16, 159, 324, 21, 3566, 10, 13824, 3439, 4584, 3890, 4851]]


In [0]:
max_len = 0;
for i in range(0, len(encoded)):
    if max_len < len(encoded[i]):
        max_len = len(encoded[i])

max_len = max_len + 1
print(max_len)
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

1404
210338


In [0]:
from keras.preprocessing.sequence import pad_sequences

padded_docs = pad_sequences(encoded, maxlen=max_len, padding='post')

print(padded_docs)
#X_train, X_test, Y_train, Y_test = train_test_split( padded_docs, Y_train, test_size=0.2, random_state=42)

[[  688    75     1 ...     0     0     0]
 [96145    52  2635 ...     0     0     0]
 [  412   437    73 ...     0     0     0]
 ...
 [34278  7330  5208 ...     0     0     0]
 [    4    11   574 ...     0     0     0]
 [    4     7   134 ...     0     0     0]]


In [0]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=42)
#X_train, Y_train = ros.fit_resample(padded_docs, Y_train)

#X_train, X_test, Y_train, Y_test = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)

X_train, X_test, Y_train, Y_test = train_test_split( padded_docs, Y_train, test_size=0.2, random_state=42)
X_train, Y_train = ros.fit_resample(X_train, Y_train)


/usr/local/lib/python3.6/dist-packages/imblearn/utils/_validation.py:93: UserWarning: 'y' should be of types ('binary', 'multiclass', 'multilabel-indicator') only. Got unknown instead.
  TARGET_KIND, type_of_target(y)))


In [0]:
embeddings_index = dict()
f = open('drive/My Drive/glove.6B.100d.txt')
for line in f:
	values = line.split()
	word = values[0]
	coefs = np.asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()

In [0]:
embedding_matrix = np.zeros((vocab_size, 100))
for word, i in tokenizer.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector

In [0]:
input_seq = Input(shape=(1404, ))
embed = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=max_len, trainable=True)(input_seq)
x = Conv1D(256, 4, activation='relu')(embed)
x = MaxPooling1D(2, 1)(x)
x = Conv1D(128, 4, activation='relu')(x)
x = GlobalMaxPool1D()(x)
x = Dense(512, activation='relu')(x)
x = Dense(128, activation='relu')(x)
classi = Dense(1, activation='sigmoid')(x)
classifier = Model(input_seq, classi)
classifier.compile(optimizer='adadelta', loss='binary_crossentropy', metrics=['accuracy'])
print(classifier.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 1404)              0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 1404, 100)         21033800  
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 1401, 256)         102656    
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 1400, 256)         0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 1397, 128)         131200    
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 128)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 512)               66048     
__________

In [0]:
classifier.fit(X_train, Y_train, batch_size=512, epochs = 30, 
    verbose=1, validation_data= (X_test, Y_test))

Train on 230836 samples, validate on 31915 samples
Epoch 1/30
230836/230836 [==============================] - 209s 906us/step - loss: 0.3020 - acc: 0.8668 - val_loss: 0.2204 - val_acc: 0.8983
Epoch 2/30
230836/230836 [==============================] - 206s 892us/step - loss: 0.1666 - acc: 0.9340 - val_loss: 0.2215 - val_acc: 0.9038
Epoch 3/30
230836/230836 [==============================] - 205s 888us/step - loss: 0.0990 - acc: 0.9655 - val_loss: 0.1649 - val_acc: 0.9579
Epoch 4/30
230836/230836 [==============================] - 205s 887us/step - loss: 0.0624 - acc: 0.9807 - val_loss: 0.1853 - val_acc: 0.9579
Epoch 5/30
230836/230836 [==============================] - 204s 885us/step - loss: 0.0461 - acc: 0.9868 - val_loss: 0.2011 - val_acc: 0.9581
Epoch 6/30
230836/230836 [==============================] - 204s 883us/step - loss: 0.0331 - acc: 0.9910 - val_loss: 0.2327 - val_acc: 0.9582
Epoch 7/30
230836/230836 [==============================] - 204s 883us/step - loss: 0.0299 - acc:

In [0]:
predictions = classifier.predict(X_test)
print(predictions)

for i in range(len(predictions)):
    if predictions[i]>0.5:
        predictions[i] = 1
    else:
        predictions[i] = 0

print(predictions)

[[2.0265579e-06]
 [0.0000000e+00]
 [0.0000000e+00]
 ...
 [2.7728801e-14]
 [9.9999774e-01]
 [2.9559948e-12]]
[[0.]
 [0.]
 [0.]
 ...
 [0.]
 [1.]
 [0.]]


In [0]:
from sklearn.metrics import confusion_matrix, classification_report

print(predictions)
Y_test = [[i] for i in Y_test]

print(Y_test)
print(classification_report(Y_test, predictions))

pd.DataFrame(
   confusion_matrix(Y_test, predictions),
   index = [['Actual', 'Actual'], ['Not Toxic', 'Toxic']],
   columns = [['Predicted', 'Predicted'], ['Not Toxic', 'Toxic']])

[[0.]
 [0.]
 [0.]
 ...
 [0.]
 [1.]
 [0.]]
[[0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [1], [0], [0], [0], [0], [1], [0], [0], [0], [0], [0], [0], [0], [0], [0], [1], [0], [1], [1], [1], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [1], [0], [1], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [1], [0], [0], [0], [0], [0], [1], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [1], [0], [0], [0], [0], [0], [1], [0], [0], [0], [0], [0], [0], [0], [1], [0], [0], [0], [0], [0], [0], [0], [0], [0], [1], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [1], [0], [0], [0], [0], [0], [0], [0], [0], [0], [1], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0

Predicted      
                 Not Toxic Toxic
Actual Not Toxic     28247   612
       Toxic           731  2325